### ChatOpenAI 사용하기
- 템플릿에 변수를 입력하여 prompt 생성
- ChatOpenAI 모델을 불러온 뒤, 모델에 prompt를 입력하여 결과 출력

In [1]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}", 

)
prompt = template.format(country_a="Metixco", country_b="Thailand")
chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)



'The distance between Metepec, Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when traveling by air.'

In [11]:

template = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your nanme?")
    ]
)

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand"
)

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

### OutputParser로 결과 출력하기
- 출력 형태를 상황에 맞게 수정할 때 사용

In [15]:
from langchain.schema import BaseOutputParser

class CommanOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommanOutputParser()

p.parse("Hello,  how,are,you")

['Hello', 'how', 'are', 'you']

In [18]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. DO NOT replay with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommanOutputParser()
p.parse(result.content)

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

### Chain 사용하기
- Template, Chat model, Output Parser를 Chain으로 엮음
- chain.invoke()를 통해 Langchain에서 템플릿 내용을 모델에 입력한 뒤 결과를 parser에 넣어 출력함

In [19]:
# Chain of template, Language Model, and Output Parser
chain = template | chat | CommanOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

### 2개의 Chain 연결하기
- Chain끼리 연결하기
- 앞 Chain의 출력을 뒤에서 입력으로 받아 결과 출력하기

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat 

In [3]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain
final_chain.invoke({
    "cuisine": "indian"
})


Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, and chili powder. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best resul

### FewShotPromptTemplate
- Few-shot learning을 위한 예제를 전달하는 방식

In [5]:
# Few-shot 아닌 방식
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

chat.predict("What do you know about France?")

France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is known for its wine production, with regions such as Bordeaux, Burgundy, and Champagne producing some of the world's most renowned wines. French cuisine is also highly regarded, with dishes such as croissants, baguettes, escargot, and coq au vin being popular both in France and around the world.

The country has a diverse landscape, ranging from the snow-capped Alps in the east to the sandy beaches of the French Riviera in the south. France is also home to numerous UNESCO World Heritage sites, including the Palace of Versailles, Mont-Saint-Michel, and the Pont du Gard.

France has a rich cultural heritage, with famous artists such as Claude Monet, 

"France is a country located in Western Europe, known for its rich history, culture, and cuisine. It is the largest country in the European Union by land area and the third-largest in Europe overall. The capital city is Paris, which is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is known for its wine production, with regions such as Bordeaux, Burgundy, and Champagne producing some of the world's most renowned wines. French cuisine is also highly regarded, with dishes such as croissants, baguettes, escargot, and coq au vin being popular both in France and around the world.\n\nThe country has a diverse landscape, ranging from the snow-capped Alps in the east to the sandy beaches of the French Riviera in the south. France is also home to numerous UNESCO World Heritage sites, including the Palace of Versailles, Mont-Saint-Michel, and the Pont du Gard.\n\nFrance has a rich cultural heritage, with famous artists such as Claude 

In [8]:
# Few-shot 방식
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country":"Australia"
})


AI:
        Here is what I know:
        Capital: Canberra
        Language: English
        Food: Vegemite, Meat Pies, and BBQ
        Currency: Australian Dollar

AIMessageChunk(content='AI:\n        Here is what I know:\n        Capital: Canberra\n        Language: English\n        Food: Vegemite, Meat Pies, and BBQ\n        Currency: Australian Dollar')